In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.chains import LLMChain

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

C:\Users\wonhy\AppData\Local\Temp\ipykernel_2864\3189218925.py:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')


In [2]:
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load_and_split()

In [3]:
def create_vector_store(_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(_docs)
    persist_directory = "./chroma_db"
    vectorstore = Chroma.from_documents(
        split_docs, 
        OpenAIEmbeddings(model='text-embedding-3-small'),
        persist_directory=persist_directory
    )
    return vectorstore

In [4]:
def add_to_vector_store(new_docs, persist_directory="./chroma_db"):
    # 1. 기존 벡터 저장소 불러오기
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
    )

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(new_docs)

    # 3. 문서 추가
    vectorstore.add_documents(split_docs)

    return vectorstore

In [5]:
path='../data/문형빈 교수님 sci'
pl=os.listdir(path)

In [6]:
mk=load_and_split_pdf(os.path.join(path, pl[0]))
create_vector_store(mk)

In [7]:
for i in pl[1:]:
    ad=load_and_split_pdf(os.path.join(path, i))
    add_to_vector_store(ad)

C:\Users\wonhy\AppData\Local\Temp\ipykernel_2864\3363452400.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [14]:
# load from disk
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "HyungBin Moon's 2021 paper?"
result = db3.similarity_search(query)
print(result[0].page_content)

levels.Int J Energy Res. 2020;44:8037–8052. https://
doi.org/10.1002/er.4974
8052 MOON AND WOO
 1099114x, 2020, 10, Downloaded from https://onlinelibrary.wiley.com/doi/10.1002/er.4974 by Pukyong National University, Wiley Online Library on [14/04/2025]. See the Terms and Conditions (https://onlinelibrary.wiley.com/terms-and-conditions) on Wiley Online Library for rules of use; OA articles are governed by the applicable Creative Commons License


In [15]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "explain hyungbin moon's 2020 paper"
docs = db3.similarity_search(query, k=15)

formatted_docs = [Document(page_content=doc.page_content) for doc in docs]

prompt_template = """
다음은 논문 일부입니다. 이 내용을 바탕으로 석사 논문 내용을 연구대상 및 방법, 연구방법, 연구결과, 제언을 핵심적으로 요약해 주세요.
논문 내용:
{context}
요약:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context"])
llm = ChatOpenAI(model="gpt-4o")
chain = LLMChain(llm=llm, prompt=PROMPT)
context_text = "\n\n".join([doc.page_content for doc in formatted_docs])
summary = chain.run(context=context_text)
print(summary, "\n")

C:\Users\wonhy\AppData\Local\Temp\ipykernel_14212\2887330060.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")
C:\Users\wonhy\AppData\Local\Temp\ipykernel_14212\2887330060.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=PROMPT)
C:\Users\wonhy\AppData\Local\Temp\ipykernel_14212\2887330060.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chain.run(context=context_text)


**연구대상 및 방법:**
본 연구는 한국 내 사용후핵연료 관리 전략에 대한 다양한 공공의 태도를 분석하는 것을 목표로 하였다. 이를 위해 2020년 1월, 한국의 20세에서 59세 사이의 성인을 대상으로 온라인 설문조사를 실시하였다. 조사 대상자는 성별, 연령, 지역 등 인구통계학적 요소를 고려하여 비례층화표본추출법에 따라 선정된 1,000명으로 구성되었다.

**연구방법:**
연구는 사용후핵연료 관리에 대한 개인 및 세그먼트 수준에서의 공공 선호를 분석하였다. 설문조사를 통해 수집된 데이터를 활용하여 통계적 분석을 수행하였으며, 수집된 데이터의 분석에는 주로 다변량 분석 기법이 활용되었다.

**연구결과:**
분석 결과, 대중은 사용후핵연료 저장소에 대해 위험성, 파괴, 고통 등 부정적인 영향을 우려하며 전문가들과 차이를 보였다. 세그먼트 2의 수용률은 85.46%에서 58.46%까지 차이를 보였으며, 전체 수용률은 86.99%에서 52.93%로 나타났다. 사용후핵연료의 깊이, 핵 재처리, 민주적 정책 과정, 전기 요금 증가 등의 요소가 대중의 선호에 영향을 미치는 것으로 나타났다.

**제언:**
본 연구에서는 지역별 기상 조건을 고려한 개별 정책의 필요성을 강조하였다. 특히, 대기오염물질 배출과 그로 인한 대기질 수준이 기상 조건에 영향을 받는다는 점에서, 정책 수립 시 지역 기상 조건의 반영이 필요하다. 또한, 공공의 인식과 수용도를 높이기 위해 보다 투명하고 민주적인 정책 과정이 요구된다. 연구는 한국 에너지기술평가원과 산업통상자원부의 지원을 받았다. 

